# header

In [1]:
'''
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}

setInterval(ConnectButton,60000);
'''

# from google.colab import drive
# from os import chdir

# drive.mount('/content/drive')
# project_path = '/content/drive/MyDrive/Gproject/MIT_glyco'
# chdir(project_path)

'\nfunction ConnectButton(){\n    console.log("Connect pushed"); \n    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() \n}\n\nsetInterval(ConnectButton,60000);\n'

In [2]:
import pandas as pd
import numpy as np

import os
from os.path import exists

print(os.getcwd()) # current working directory
update = False

D:\Gdrive\My Drive\Gproject\MIT_glyco


In [3]:
from src.Data import *
from src.Preprocessing import *

## Mauli's dataset

In [4]:
# load O-GlcNAcylated site data
columns = ['protein', 'positive_site', 'sequence']
mauli_sites = pd.read_csv("./OGP-master/all_sites.csv", 
                          header=None, names=['protein', 'positive_site', 'sequence'])
protein_list = list(mauli_sites.protein.unique()) # all kinds of proteins in Mauli's dataset
display(mauli_sites)

,protein,positive_site,sequence
0,A2ABU4,430,MTLPHSPGSAGEPQASQTVQVHRLEHRQEEEQKEERQHSLQMGSSV...
1,A2AGT5,568,MGDDSEWLKLPVDQKCEHKLWKARLSGYEEALKIFQKIKDEKSPEW...
2,A2AHJ4,264,MAAAPTQIEAELYYLIARFLQSGPCNKSAQVLVQELEEHQLIPRRL...
3,A2AHJ4,279,MAAAPTQIEAELYYLIARFLQSGPCNKSAQVLVQELEEHQLIPRRL...
4,A2AKB9,87,MFPFGPHSPGGDETAGAEEPPPLGGPAAASRPPSPAPRPASPQRGA...
...,...,...,...
531,Q9UPN6,614,MEAVKTFNSELYSLNDYKPPISKAKMTQITKAAIKAIKFYKHVVQS...
532,Q9UQ35,2236,MYNGIGLPTPRGSGTNGYVQRNLSLVRGRRGERPDYKGEEELRRLE...
533,Q9Y2X9,891,MKIGSGFLSGGGGTGSSGGSGSGGGGSGGGGGGGSSGRRAEMEPTF...
534,Q9Y520,2693,MSEKSGQSTKAKDGKKYATLSLFNTYKGKSLETQKTTARHGLQSLG...


## grouping positive sites

In [5]:
##########
save_path = f'./data/group_all_sites.csv'
new_columns = ['protein', 'positive_sites', 'sequence']
group_sites = pd.DataFrame([], columns=new_columns)
##########
if not exists(save_path) or update:
    for name in protein_list:
        protein = mauli_sites[mauli_sites['protein']==name]
        sites = [x for x in protein.positive_site.values]
        sequence = protein.sequence.iloc[0]
        temp_df = pd.DataFrame([[name, sites, sequence]], columns = new_columns)
        group_sites = pd.concat([group_sites, temp_df], axis=0)
        
    group_sites = group_sites.reset_index(drop=True)
    group_sites.to_csv(save_path, index=False)
else:
    group_sites = pd.read_csv(save_path)
    
display(group_sites)

,protein,positive_sites,sequence
0,A2ABU4,[430],MTLPHSPGSAGEPQASQTVQVHRLEHRQEEEQKEERQHSLQMGSSV...
1,A2AGT5,[568],MGDDSEWLKLPVDQKCEHKLWKARLSGYEEALKIFQKIKDEKSPEW...
2,A2AHJ4,"[264, 279]",MAAAPTQIEAELYYLIARFLQSGPCNKSAQVLVQELEEHQLIPRRL...
3,A2AKB9,"[87, 99, 100, 364]",MFPFGPHSPGGDETAGAEEPPPLGGPAAASRPPSPAPRPASPQRGA...
4,E9Q1P8,[208],MAAAVAVAAASRRQSCYLCDLPRMPWAMIWDFTEPVCRGCVNYEGA...
...,...,...,...
270,Q9P2N5,[738],MLIEDVDALKSWLAKLLEPICDADPSALANYVVALVKKDKPEKELK...
271,Q9UPN6,"[615, 614]",MEAVKTFNSELYSLNDYKPPISKAKMTQITKAAIKAIKFYKHVVQS...
272,Q9UQ35,[2236],MYNGIGLPTPRGSGTNGYVQRNLSLVRGRRGERPDYKGEEELRRLE...
273,Q9Y2X9,[891],MKIGSGFLSGGGGTGSSGGSGSGGGGSGGGGGGGSSGRRAEMEPTF...


## concatenation of 2 different data

1. dynamine_results: flexibility
2. spider3_result: secondary structure

In [6]:
flex_path = "./OGP-master/dynamine_results" # directory of flexibility results
angle_path = "./OGP-master/spider3_results" # directory of angle related information

pass_list = call_pass_list()

for x in pass_list:
    protein_list.remove(x) 
    
save_path = './data/concatenated_dataset'
for name in protein_list:
    save_name = f"{save_path}/{name}.csv"
    if not exists(save_name) or update:
        # load flexibility data
        flex_data = pd.read_fwf(f"{flex_path}/{name}_backbone.pred", header=None)
        flex_data.columns = ["values"]
        flex_data = flex_data[11:].reset_index(drop=True)
        flex_data['SEQ'] = flex_data['values'].apply(lambda x: x.split()[0])
        flex_data['flexibility'] = flex_data['values'].apply(lambda x: x.split()[1])
        flex_data = flex_data[['SEQ', 'flexibility']]
        
        # load angle data
        angle_data = pd.read_csv(f"{angle_path}/{name}.spd33")
        # split values from raw angle data
        angle_columns = angle_data.columns[0].split()
        angle_data.columns = ["values"]
        for c, column in enumerate(angle_columns):
            angle_data[column] = angle_data['values'].apply(lambda x: x.split()[c])
            
        # concatenate and add a feature of positivie sites 
        sencondary_data = pd.concat([angle_data.iloc[:,1:], flex_data.iloc[:,1:]], axis=1)
        sencondary_data.index.name = name
        
        # save the dataframe
        sencondary_data.to_csv(save_name, index=True)
        
    else:
        sencondary_data = pd.read_csv(save_name)
        
display(sencondary_data)

,Q9Y520,#,SEQ,SS,ASA,Phi,Psi,Theta(i-1=>i+1),Tau(i-2=>i+2),HSE_alpha_up,HSE_alpha_down,P(C),P(H),P(E),flexibility
0,0,1,M,C,155.0,-95.8,133.1,115.4,165.4,3.1,3.3,0.988,0.007,0.005,0.422
1,1,2,S,C,103.7,-97.1,140.5,116.6,-157.5,1.4,3.8,0.952,0.018,0.029,0.402
2,2,3,E,C,151.2,-89.7,125.5,111.2,-135.5,2.9,7.7,0.918,0.031,0.051,0.414
3,3,4,K,C,151.6,-89.7,115.7,109.8,-160.5,3.5,6.9,0.915,0.038,0.047,0.434
4,4,5,S,C,92.5,-87.4,115.3,109.1,-128.5,2.4,7.0,0.939,0.034,0.028,0.453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2891,2891,2892,E,C,110.1,-95.9,129.5,114.1,-151.5,8.1,12.4,0.686,0.253,0.062,0.580
2892,2892,2893,E,C,129.9,-89.6,124.2,111.0,-140.1,5.4,11.0,0.713,0.227,0.061,0.540
2893,2893,2894,T,C,106.7,-91.4,100.3,109.1,-163.0,3.1,9.1,0.753,0.168,0.080,0.473
2894,2894,2895,K,C,160.4,-89.3,66.8,107.1,136.5,2.7,7.7,0.902,0.057,0.042,0.420


## data for original models

In [7]:
### data processing ###
### Make dataset for machine learning model ###
### execution time: 1m 57s ###

for name in protein_list:
    load_path = f"./data/concatenated_dataset/{name}.csv"
    save_path = f"./data/data_for_ml/{name}.csv"
    if not exists(save_path) or update:
        machine_dataset = pd.read_csv(load_path, index_col=0)
        
        # determine side chain
        side_window = [-1, 5]
        for side in range(side_window[0], side_window[1]+1):
            if side < 0:
                machine_dataset[f"side_{side}"] = pd.concat([pd.Series(np.zeros(abs(side))), machine_dataset.SEQ.iloc[:side]]).apply(side_chain).reset_index(drop=True)
            elif side > 0:
                machine_dataset[f"side_{side}"] = pd.concat([machine_dataset.SEQ.iloc[side:], pd.Series(np.zeros(abs(side)))]).apply(side_chain).reset_index(drop=True)
            else:
                pass
            
        # count non-polar aliphatic amino residues
        machine_dataset['nAli'] = machine_dataset['#'].apply(lambda x: nonpolar_aliphatic(machine_dataset, x-1, start=-3, end=-1))
        
        # count positively charged residues
        machine_dataset['nPos'] = machine_dataset['#'].apply(lambda x: positively_charged(machine_dataset, x-1, start=-7, end=-5))
        
        # count serine or threonine
        machine_dataset['nS/nT'] = machine_dataset['#'].apply(lambda x: S_and_T(machine_dataset, x-1, start=-10, end=10))
        
        # determine presence of proline after +1 position
        machine_dataset['Proline'] = machine_dataset['#'].apply(lambda x: is_proline_after(machine_dataset, x-1, after=1))
        
        # determine secondary structure using Phi-Psi
        machine_dataset['phi_psi'] = machine_dataset['#'].apply(lambda x: phi_psi(machine_dataset, x-1))
        
        # positivity
        machine_dataset['positivity'] = 0
        sites = pd.read_csv(f"./data/group_all_sites.csv")
        sites = sites[sites['protein']==name].positive_sites.iloc[0]
        sites_index = [int(x)-1 for x in sites[1:-1].split(', ')]
        machine_dataset.loc[sites_index, "positivity"] = 1
        
        # save file
        machine_dataset.to_csv(save_path, index=True)
        
    else:
        machine_dataset = pd.read_csv(save_path, index_col=0)
        
display(machine_dataset)

,#,SEQ,SS,ASA,Phi,Psi,Theta(i-1=>i+1),Tau(i-2=>i+2),HSE_alpha_up,HSE_alpha_down,...,side_2,side_3,side_4,side_5,nAli,nPos,nS/nT,Proline,phi_psi,positivity
Q9Y520,,,,,,,,,,,,,,,,,,,,,
0,1,M,C,155.0,-95.8,133.1,115.4,165.4,3.1,3.3,...,normal,long,small,gly,0,0,4,0,alpha,0
1,2,S,C,103.7,-97.1,140.5,116.6,-157.5,1.4,3.8,...,long,small,gly,normal,0,0,4,0,alpha,0
2,3,E,C,151.2,-89.7,125.5,111.2,-135.5,2.9,7.7,...,small,gly,normal,small,0,0,4,0,alpha,0
3,4,K,C,151.6,-89.7,115.7,109.8,-160.5,3.5,6.9,...,gly,normal,small,small,0,0,4,0,alpha,0
4,5,S,C,92.5,-87.4,115.3,109.1,-128.5,2.4,7.0,...,normal,small,small,long,0,0,4,0,alpha,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2891,2892,E,C,110.1,-95.9,129.5,114.1,-151.5,8.1,12.4,...,small,long,small,None,1,1,3,0,alpha,0
2892,2893,E,C,129.9,-89.6,124.2,111.0,-140.1,5.4,11.0,...,long,small,None,None,0,0,3,0,alpha,0
2893,2894,T,C,106.7,-91.4,100.3,109.1,-163.0,3.1,9.1,...,small,None,None,None,0,0,3,0,alpha,0


In [8]:
save_path = f"./data/data_for_ml.csv"

if not exists(save_name) or update:
    for i, name in enumerate(protein_list):
        load_name = f"./data/data_for_ml/{name}.csv"
        temp = pd.read_csv(load_name, index_col=0)
        temp['protein'] = temp.index.name

        if i==0:
            dataset = temp
        else:
            dataset = pd.concat([dataset, temp], axis=0)

    dataset = dataset.reset_index(drop=True)
    dataset.to_csv(save_path, index=False)
else:
    dataset = pd.read_csv(save_path)
    
display(dataset)

,#,SEQ,SS,ASA,Phi,Psi,Theta(i-1=>i+1),Tau(i-2=>i+2),HSE_alpha_up,HSE_alpha_down,...,side_3,side_4,side_5,nAli,nPos,nS/nT,Proline,phi_psi,positivity,protein
0,1,M,C,112.7,-100.9,139.3,119.5,165.0,8.5,9.1,...,pro,cycle,small,0,0,3,0,alpha,0,A2ABU4
1,2,T,C,103.3,-102.0,132.1,117.6,-150.0,3.8,13.9,...,cycle,small,pro,0,0,3,0,alpha,0,A2ABU4
2,3,L,C,50.9,-97.8,134.7,118.5,-149.2,16.4,11.6,...,small,pro,gly,0,0,3,1,alpha,0,A2ABU4
3,4,P,C,77.2,-69.2,144.0,111.0,-105.3,7.5,16.7,...,pro,gly,small,1,0,3,0,alpha,0,A2ABU4
4,5,H,C,80.3,-95.4,141.5,118.6,-135.6,13.3,13.3,...,gly,small,very_small,2,0,3,0,alpha,0,A2ABU4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257573,2892,E,C,110.1,-95.9,129.5,114.1,-151.5,8.1,12.4,...,long,small,None,1,1,3,0,alpha,0,Q9Y520
257574,2893,E,C,129.9,-89.6,124.2,111.0,-140.1,5.4,11.0,...,small,None,None,0,0,3,0,alpha,0,Q9Y520
257575,2894,T,C,106.7,-91.4,100.3,109.1,-163.0,3.1,9.1,...,None,None,None,0,0,3,0,alpha,0,Q9Y520
257576,2895,K,C,160.4,-89.3,66.8,107.1,136.5,2.7,7.7,...,None,None,None,0,1,3,0,other,0,Q9Y520


In [9]:
dataset.describe()

,#,ASA,Phi,Psi,Theta(i-1=>i+1),Tau(i-2=>i+2),HSE_alpha_up,HSE_alpha_down,P(C),P(H),P(E),flexibility,nAli,nPos,nS/nT,Proline,positivity
count,257578.000000,257578.000000,257578.000000,257578.00000,257578.000000,257578.000000,257578.000000,257578.000000,257578.000000,257578.000000,257578.000000,257578.000000,257578.000000,257578.000000,257578.000000,257578.000000,257578.000000
mean,860.577111,62.125352,-75.621042,44.99207,106.016234,-16.500547,12.032931,17.971759,0.568390,0.307722,0.123877,0.747209,0.995667,0.410788,3.345200,0.074277,0.002054
std,859.700289,36.664497,39.776313,80.41562,10.639836,107.375685,6.803528,5.146543,0.363292,0.374990,0.229347,0.114476,0.831814,0.613402,2.261084,0.262221,0.045272
min,1.000000,0.100000,-180.000000,-180.00000,86.000000,-180.000000,0.400000,2.500000,0.000000,0.000000,0.000000,0.212000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,246.000000,33.600000,-92.900000,-33.10000,95.300000,-130.300000,6.500000,14.300000,0.166000,0.018000,0.002000,0.673000,0.000000,0.000000,2.000000,0.000000,0.000000
50%,564.000000,61.100000,-79.800000,17.70000,105.300000,49.900000,10.700000,17.500000,0.705000,0.103000,0.031000,0.756000,1.000000,0.000000,3.000000,0.000000,0.000000
75%,1199.000000,88.500000,-67.200000,131.40000,114.300000,58.400000,16.600000,21.400000,0.892000,0.574000,0.111000,0.828000,2.000000,1.000000,5.000000,0.000000,0.000000
max,5068.000000,207.700000,180.000000,180.00000,150.400000,180.000000,30.500000,33.400000,1.000000,1.000000,1.000000,1.207000,3.000000,3.000000,21.000000,1.000000,1.000000


## data for augmented models

In [9]:
match = [exists(f"./data/from_Krishna/{x}") for x in protein_list]
dict_match = dict(zip(protein_list, match))
dict_match

{'A2ABU4': True,
 'A2AGT5': False,
 'A2AHJ4': True,
 'A2AKB9': True,
 'E9Q1P8': True,
 'E9Q5G3': True,
 'F6T0L5': False,
 'F6TYF8': False,
 'O35161': False,
 'O70263': True,
 'O70400': True,
 'P0CG49': False,
 'P14602': True,
 'P20152': True,
 'P23927': True,
 'P25446': True,
 'P26039': True,
 'P31001': True,
 'P31230': True,
 'P35279': False,
 'P42128': True,
 'P47746': True,
 'P48614_2': False,
 'P48678': True,
 'P53668': True,
 'P59759': True,
 'P60710': False,
 'P63017': True,
 'P63248': False,
 'P68033': True,
 'P70402': True,
 'P70670': True,
 'P83741': True,
 'P97306': True,
 'P97326': True,
 'P97379': True,
 'P97855': True,
 'Q01705_2': False,
 'Q03173_4': False,
 'Q04690_4': False,
 'Q3TN34': True,
 'Q3UCQ1': True,
 'Q3USJ8': True,
 'Q4U2R1_2': False,
 'Q505D9': True,
 'Q571K4': True,
 'Q5BJ29': True,
 'Q5DTJ9': True,
 'Q5DU31': True,
 'Q62381_2': False,
 'Q62417_5': False,
 'Q62418': True,
 'Q62419': True,
 'Q62523': True,
 'Q66L42': True,
 'Q69ZI1_3': False,
 'Q6GQX2': True,

In [10]:
import shutil

target_folder = './data/from_AlphaFold'
if not exists(target_folder):
    os.makedirs(target_folder)

# copy files from Krishna folder to AlphaFold folder
protein_match = [x for x in protein_list if exists(f"./data/from_Krishna/{x}") ]
print("total number of matching proteins:", len(protein_match))
for name in protein_match:
    src_path = f'./data/from_Krishna/{name}/all_AA/combined-output-all-residues-features.csv'
    tgt_path = f'{target_folder}/{name}.csv'
    if not exists(tgt_path) or update:
        shutil.copyfile(src_path, tgt_path)
        
# {Krishna name : Mauli name}
protein_names = {
    'A0A024RAY2':'P05783',    # 1
    'P0CG62':'P0CG49',        # 2
    'P63249':'P63248',        # 3
    'P68406':'P24622_2',      # 4
    'Q4R561_P60710':'P60710', # 5
    'Q9WVB1':'P35279',        # 6
}
print("total number of name-mismatching proteins:", len(protein_names))
for Krishna, Mauli in protein_names.items():
    src_path = f'./data/from_Krishna/{Krishna}/all_AA/combined-output-all-residues-features.csv'
    tgt_path = f'{target_folder}/{Mauli}.csv'
    if Mauli in protein_list:
        if not exists(tgt_path) or update:
            shutil.copyfile(src_path, tgt_path)   
#         print(f'protein from {Krishna} to {Mauli}')
    else:
        print(f'**warning** protein from {Krishna} to {Mauli} does not exist')
        
missing_list = [
    'Q80TE4',
    'Q80VP0',
    'A2ABU4',
    'P07756',
    'Q5DTJ9',
    'Q6GQX2',
    'Q6ZPS6',
    'Q8CDG3',
    'Q8CG79',
    'Q66L42',
    'Q76LL6'
]

for name in missing_list:
    src_path = f'./data/from_Krishna/combined-output-all-residues-features_{name}.csv'
    tgt_path = f'{target_folder}/{name}.csv'
    shutil.copyfile(src_path, tgt_path)

total number of matching proteins: 88
total number of name-mismatching proteins: 6
**warning** protein from P68406 to P24622_2 does not exist


In [11]:
protein_augmented = [x for x in protein_list if exists(f"./data/from_AlphaFold/{x}.csv")]
print('total number of proteins for data augmentation:', len(protein_augmented))

target_folder = f"./data/data_for_ml(augmented)"
if not exists(target_folder):
    os.makedirs(target_folder)

charge_columns = pd.read_csv('./data/from_Krishna/features-all-names.csv').name.to_list()
count = 0
for name in protein_augmented:
    save_path = f'{target_folder}/{name}.csv'
    if not exists(save_path) or update:
        load_path = f'./data/from_AlphaFold/{name}.csv'
        charge_data = pd.read_csv(load_path, sep=' ', header=None)
        charge_data.columns = charge_columns[:-1]
        charge_data = charge_data.sort_values(by='#').reset_index(drop=True)
        charge_data = charge_data.iloc[:,1:] # remove '#' column to prevent duplicated columns
        
        load_path = f'./data/data_for_ml/{name}.csv'
        data = pd.read_csv(load_path, index_col=0)
        
        # concatenate
        aug_data = pd.concat([data, charge_data], axis=1) 
        if aug_data.isnull().sum().sum():
            print(f'protein {name} has null values in augmented dataset')
            count += 1
        else:
            aug_data.to_csv(save_path, index=False)
    
    else:
        try:
            aug_data = pd.read_csv(save_path)
            
        except:
            print(f'protein {name} has null values in augmented dataset')
            count += 1
print(f'total number of augmented proteins: {len(protein_augmented)-count}')
display(aug_data)

total number of proteins for data augmentation: 93
protein P26039 has null values in augmented dataset
protein P70670 has null values in augmented dataset
protein P83741 has null values in augmented dataset
protein A2AQ25 has null values in augmented dataset
total number of augmented proteins: 89


,#,SEQ,SS,ASA,Phi,Psi,Theta(i-1=>i+1),Tau(i-2=>i+2),HSE_alpha_up,HSE_alpha_down,...,exposed_positive_charge_all_with_around_target_ser_thr,exposed_positive_charge_all_backbone_around_target_ser_thr,exposed_positive_charge_all_sidechain_around_target_ser_thr,exposed_positive_charge_all_backbone_with_around_target_ser_thr,exposed_positive_charge_all_sidechain_with_around_target_ser_thr,exposed_negative_charge_all_around_target_ser_thr,exposed_negative_charge_all_with_around_target_ser_thr,exposed_negative_charge_all_backbone_around_target_ser_thr,exposed_negative_charge_all_sidechain_around_target_ser_thr,exposed_negative_charge_all_backbone_with_around_target_ser_thr
0,1,M,C,127.7,-98.3,139.6,118.2,162.6,6.8,9.2,...,0.00,1.52,0.51,2.15,-4.21,-5.65,-1.78,-2.43,-2.59,-3.06
1,2,A,C,71.3,-81.6,142.2,115.5,-110.9,7.9,10.8,...,1.53,3.43,1.53,3.70,-6.95,-8.20,-3.10,-3.85,-4.08,-4.12
2,3,D,C,73.7,-87.2,126.7,113.1,-121.0,9.2,13.9,...,2.04,3.91,2.04,4.71,-8.51,-10.82,-4.04,-4.47,-4.55,-6.27
3,4,P,C,67.1,-63.3,-30.6,97.2,-125.9,11.3,13.5,...,1.53,7.60,2.04,8.05,-12.27,-13.43,-4.26,-8.01,-5.06,-8.37
4,5,R,C,115.6,-65.0,-36.0,93.7,59.5,8.5,14.8,...,2.04,4.63,2.55,8.29,-10.83,-14.47,-4.29,-6.54,-5.27,-9.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,729,A,H,29.7,-73.2,-30.4,98.8,56.7,15.8,16.9,...,4.42,8.10,4.93,8.37,-14.84,-16.09,-8.30,-6.54,-9.28,-6.81
729,730,K,H,102.8,-72.5,-24.6,97.3,56.5,9.2,17.4,...,3.91,6.35,4.42,8.19,-16.23,-17.40,-9.32,-6.91,-9.83,-7.57
730,731,Q,C,91.3,-85.0,-2.2,100.0,55.6,9.1,16.1,...,3.40,5.34,3.91,6.78,-10.20,-11.88,-6.81,-3.39,-7.32,-4.56
731,732,G,C,50.1,79.5,4.9,101.0,-84.7,4.8,17.1,...,2.89,5.26,3.40,5.35,-9.63,-10.63,-5.34,-4.29,-6.34,-4.29
